# RAG Pipeline with BGE and Llama 3
---

In this notebook, we build a Retrieval-Augmented Generation (RAG) pipeline that enhances a language model with up-to-date news information. We use **BAAI/bge-base-en-v1.5** to generate embeddings and retrieve the most relevant BBC news articles based on a user query. The retrieved documents are then injected into a prompt sent to **meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo**, a model trained on data up to December 2023.

By combining semantic retrieval with generation, the system enables the LLM to produce responses grounded in recent events (including 2024), improving factual accuracy and contextual relevance.

# Table of Contents
- [ 1 - Introduction](#1)
  - [ 1.1 RAG architecture overview](#1-1)
  - [ 1.2 Importing the necessary libraries](#1-2)
- [ 2 - Loading the dataset](#2)
- [ 3 - Embedding generation](#3)
  - [ 3.1 Loading the Embedding Model](#3-1)
  - [ 3.2 Generating Document Embeddings](#3-2)
  - [ 3.3 Generating the final Prompt](#3-3)
- [ 4 - LLM calls](#4)

<a id='1'></a>
## 1 - Introduction

---

<a id='1-1'></a>
### 1.1 RAG Architecture Overview

Below is a simplified representation of a Retrieval-Augmented Generation (RAG) pipeline:

<div align="center">
  <img src="C:/Users/berka/OneDrive/Bureau/sysrag/src/assets/rag_overview.png" alt="RAG Overview" width="60%">
</div>

The system follows a structured workflow. A retriever first identifies the most relevant documents from the dataset based on a user query. The retrieved content is then formatted and injected into an augmented prompt. This enriched prompt is finally passed to the language model to generate a grounded response.

To evaluate the impact of retrieval, responses generated with the RAG pipeline are compared against responses produced without additional retrieved context. This comparison highlights how external knowledge influences factual accuracy and relevance in the model’s output.

<a id='1-2'></a>
### 1.2 Importing the necessary libraries

In [1]:
import os
import sys
from pathlib import Path
sys.path.extend([
    str(Path.cwd().parent),
    str(Path.cwd().parent / "src"),
])
import data
from sentence_transformers import SentenceTransformer
from utils.formatting import (
    pprint_json,
    read_dataframe,
    format_relevant_data,
)
from utils.rag_core import (
    query_news,
    build_embeddings_joblib,
    retrieve,
    get_relevant_data,
    generate_final_prompt,
    generate_with_single_input,
    generate_with_multiple_input,
    llm_call,
    display_widget,
)

<a id='2'></a>
## 2 - Loading the dataset

In [2]:
NEWS_DATA = read_dataframe(path=os.path.join(os.path.dirname(data.__file__), "news_data_dedup.csv"))

In [3]:
pprint_json(NEWS_DATA[9:11])

[
  {
    "guid": "5dae28f191cfd1047f67c409e616fc3f",
    "title": "Paris's Moulin Rouge loses windmill sails overnight",
    "description": "The cause of the sails' collapse from the roof of the world famous cabaret club is not yet clear.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-europe-68895836",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  },
  {
    "guid": "d2c3ff79d4e068911d05416ca061cd51",
    "title": "Ukraine uses longer-range US missiles for first time",
    "description": "Missiles secretly delivered this month have been used to strike Russian targets in Crimea, US media say.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-europe-68893196",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  }
]


Important fields are `title`, `description`, `url` and `published_at`. These fields will give good information to the LLM to answer the majority of questions with good enough data.

In [4]:
indices = [3, 6, 9]
pprint_json(query_news(indices=indices, dataset=NEWS_DATA))

[
  {
    "guid": "e696224ac208878a5cec8bdc9f97c632",
    "title": "Europe risks dying and faces big decisions - Macron",
    "description": "The French president delivers a stark warning for Europe to act fast to survive in a changing world.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-europe-68898887",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  },
  {
    "guid": "4f585bad8f61b715fbafe2f022ab0ae8",
    "title": "Supreme Court divided on whether Trump has immunity",
    "description": "The justices discussed immunity, coups, pardons, Operation Mongoose - and the future of democracy.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-us-canada-68901817",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  },
  {
    "guid": "5dae28f191cfd1047f67c409e616fc3f",
    "title": "Paris's Moulin Rouge loses windmill sails overnight",
    "description": "The cause of the sails' collapse from the roof of the world famou

<a id='3'></a>
## 3 - Embedding generation

<a id='3-1'></a> 
### 3.1 Loading the Embedding Model

In [5]:
model = SentenceTransformer("BAAI/bge-base-en-v1.5")

Loads the **[BAAI/bge-base-en-v1.5](https://huggingface.co/BAAI/bge-base-en-v1.5)** embedding model to generate 768-dimensional semantic text embeddings.

<a id='3-2'></a> 
### 3.2 Generating Document Embeddings

In [6]:
EMBEDDINGS = build_embeddings_joblib(
    dataset=NEWS_DATA,
    model=model,
    output_path="embeddings.joblib",   
    fields=["title", "description", "url", "published_at", "updated_at"],   
    batch_size=32,
    normalize_embeddings=True
)

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

<a id='3-3'></a> 
### 3.3 Generating the final Prompt

In [7]:
indices = retrieve(query="Concerts in North America", model=model, embeddings=EMBEDDINGS, top_k = 1)

In [8]:
indices

[350]

In [9]:
query = "Greatest storms in the US"
relevant_data = get_relevant_data(query=query, model=model, embeddings=EMBEDDINGS, dataset=NEWS_DATA, top_k = 1)
pprint_json(relevant_data)

[
  {
    "guid": "3ca548fe82c3fcae2c4c0c635d03eb2e",
    "title": "Large tornado seen touching down in Nebraska",
    "description": "Severe and powerful storms have moved across several US states, leaving many experiencing power shortages.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-us-canada-68860070",
    "published_at": "2024-04-26",
    "updated_at": "2024-04-28"
  }
]


In [10]:
print(generate_final_prompt(query="Tell me about the US GDP in the past 3 years.", model=model, embeddings=EMBEDDINGS, dataset=NEWS_DATA, top_k=5, use_rag=True, prompt=None))

Answer the user query below. There will be provided additional information for you to compose your answer. The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, you should not rely only on this information to answer the query, but add it to your overall knowledge.Query: Tell me about the US GDP in the past 3 years.
2024 News: {
  "guid": "60adcbc18cfa8fee177fbe0f25dd350c",
  "title": "America's Economy Is No. 1. That Means Trouble",
  "description": "If you want a single number to capture America’s economic stature, here it is: This year, the U.S. will account for 26.3% of the global gross domestic product, the highest in almost two decades. That’s based on the latest projections from the International Monetary Fund. According to the IMF, Europe’s share of world GDP has dropped 1.4 percentage points since 2018, and Japan’s by 2.1 points. The U.S. share, by contrast, is up 2.3 points.",
  "venue": "WSJ",
  "url": "https://ww

<a id='4'></a>
## 4 - LLM calls

In [11]:
output = generate_with_single_input(
    prompt="What is the capital of France?",
    together_api_key=os.getenv("TOGETHER_API_KEY")
    
)
print("Role:", output['role'])
print("Content:", output['content'])

Role: assistant
Content: The capital of France is Paris.


In [12]:
messages = [
    {'role': 'user', 'content': 'Hello, who won the FIFA world cup in 2018?'},
    {'role': 'assistant', 'content': 'France won the 2018 FIFA World Cup.'},
    {'role': 'user', 'content': 'Who was the captain?'}
]

output = generate_with_multiple_input(
    messages=messages,
    together_api_key=os.getenv("TOGETHER_API_KEY"),
    max_tokens=100
)

print("Role:", output['role'])
print("Content:", output['content'])

Role: assistant
Content: The captain of the French team that won the 2018 FIFA World Cup was Hugo Lloris.


In [13]:
query = "Tell me about the US GDP in the past 3 years."

In [14]:
print(llm_call(query="query", model=model, embeddings=EMBEDDINGS, dataset=NEWS_DATA, top_k=5, use_rag=True, prompt=None))

Based on the provided news articles from 2024, I'll compose an answer to the query. However, please note that the query itself is not provided, so I'll create a general response based on the news articles.

It appears that 2024 has been a year of significant global events, including sports, politics, and environmental concerns. In the world of sports, Liverpool's title hopes seem to be slipping away, with a point earned by West Ham against them, leaving Liverpool two points off the top of the Premier League table.

In the realm of technology, TikTok's Chinese parent firm has stated that it has no plans to sell the popular video app, despite the US passing a law that could force the sale of the app or ban it in America.

The conflict in Gaza has also been a major news story, with the BBC reporting on the challenges of finding missing loved ones in mass graves.

In the United States, the Supreme Court is debating Trump's immunity claim, with justices pushing lawyers for both sides on whe

In [15]:
display_widget(llm_call_func=llm_call, model=model, embeddings=EMBEDDINGS, dataset=NEWS_DATA, top_k=5, use_rag=True, prompt=None)

HTML(value='\n    <style>\n        .custom-output {\n            background-color: #f9f9f9;\n            color…

Text(value='', description='Query:', layout=Layout(width='100%'), placeholder='Type your query here')

Textarea(value='', description='Augmented prompt layout:', layout=Layout(height='100px', width='100%'), placeh…

IntSlider(value=5, description='Top K:', max=20, min=1, style=SliderStyle(description_width='initial'))

Button(description='Get Responses', style=ButtonStyle(button_color='#f0f0f0'))

Output()